In [2]:
import icechunk
import zarr

## Test we can open the store

In [2]:
# When opening the repo, use None for anonymous/public access
credentials = icechunk.containers_credentials({
    "gs://ismip6/": None  # None uses anonymous credentials if allowed
})
config = icechunk.RepositoryConfig.default()

icechunk_storage = icechunk.gcs_storage(bucket="ismip6-icechunk", prefix="12042025-test", from_env=True)
repo = icechunk.Repository.open(icechunk_storage, config, authorize_virtual_chunk_access=credentials)
session = repo.readonly_session(branch="main")

root = zarr.open(session.store, mode='r')

## Test we can read from the store

In [40]:
# Get the array
prefix = '/AWI_PISM1/ctrl_proj_std/base'
time_arr, x_arr, y_arr = root[f'{prefix}/time'], root[f'{prefix}/x'], root[f'{prefix}/y']

# Check the array's attributes
print(time_arr[0:10])
print(x_arr[0:10])
print(y_arr[0:10])

# Get the array
base_arr = root['/AWI_PISM1/ctrl_proj_std/base/base']

# Check the array's attributes
print(base_arr[0,0:10,0])

[4015. 4380. 4745. 5110. 5475. 5840. 6205. 6570. 6935. 7300.]
[-3040000. -3032000. -3024000. -3016000. -3008000. -3000000. -2992000.
 -2984000. -2976000. -2968000.]
[-3040000. -3032000. -3024000. -3016000. -3008000. -3000000. -2992000.
 -2984000. -2976000. -2968000.]
[1.e+20 1.e+20 1.e+20 1.e+20 1.e+20 1.e+20 1.e+20 1.e+20 1.e+20 1.e+20]


/home/jupyter/conda_env/python313/lib/python3.13/site-packages/zarr/codecs/numcodecs/_codecs.py:141: ZarrUserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


## Test we can open with xarray.open_datatree and has expected number of leaves

In [47]:
dt = xr.open_datatree(session.store, engine="zarr")
len(dt.leaves)

## Test we can plot an array

In [26]:
base_array = dt['/AWI_PISM1/ctrl_proj_std/base'].base
base_array

/home/jupyter/conda_env/python313/lib/python3.13/site-packages/zarr/codecs/numcodecs/_codecs.py:141: ZarrUserWarning: Numcodecs codecs are not in the Zarr version 3 specification and may not be supported by other zarr implementations.
  super().__init__(**codec_config)


<xarray.DataArray 'base' (time: 86, y: 761, x: 761)> Size: 199MB
[49804406 values with dtype=float32]
Coordinates:
  * time     (time) object 688B 2016-01-01 00:00:00 ... 2101-01-01 00:00:00
  * y        (y) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
  * x        (x) float64 6kB -3.04e+06 -3.032e+06 ... 3.032e+06 3.04e+06
Attributes:
    standard_name:  base_altitude
    units:          m

In [27]:
vmin, vmax = base_array.quantile([0.01, 0.99]).values
vmin, vmax

(np.float64(-1410.220947265625), np.float64(1802.943603515625))

In [34]:
base_array.hvplot.image(
    x='x', 
    y='y', 
    clim=(vmin, vmax), 
    cmap='RdBu',
    groupby='time',
    widget_location='bottom',
    aspect='equal',
    title=(" ").join(base_array.attrs['standard_name'].split("_"))
)

Column
    [0] HoloViews(DynamicMap, sizing_mode='fixed', widget_location='bottom')
    [1] WidgetBox(align=('center', 'end'))
        [0] DiscreteSlider(name='time', options={'2016-01-01 00:00:00': cf...}, value=cftime.DatetimeNoLeap(2016...)